In [1]:
import glob
import os
import re
import random
import pandas as pd
import re
import nltk
import time
import string
from nltk.corpus import stopwords

In [2]:
from PIL import Image
import pytesseract
import sys
from pdf2image import convert_from_path

In [3]:
def pdf_image(PDF_file):
    
    # Store all the pages of the PDF in a variable
    pages = convert_from_path(PDF_file, 500)
    # Counter to store images of each page of PDF to image
    print(len(pages))
    image_counter = 0
    # Iterate through all the pages stored above
    for page in pages:
        # Declaring filename for each page of PDF as JPG
        # For each page, filename will be:
        # PDF page 1 -> page_1.jpg
        # PDF page 2 -> page_2.jpg
        # PDF page 3 -> page_3.jpg
        # ....
        # PDF page n -> page_n.jpg
        filename = "test_"+str(image_counter)+".jpg"
        # Save the image of the page in system
        page.save(filename, 'JPEG')
        # Increment the counter to update filename
        image_counter = image_counter + 1
    return image_counter

In [4]:
def image_ocr(dir_path, filelimit):
    # Creating a text file to write the output
    ocr_file = "".join((dir_path,"/ocr.txt"))
    # Open the file in append mode so that 
    # All contents of all images are added to the same file
    f = open(ocr_file, "w")
    # Iterate from 1 to total number of pages
    for i in range(0, filelimit):

        # Set filename to recognize text from
        # Again, these files will be:
        # page_1.jpg
        # page_2.jpg
        # ....
        # page_n.jpg
        filename = "test_"+str(i)+".jpg"

        # Recognize the text as string in image using pytesserct
        text = str(((pytesseract.image_to_string(Image.open(filename)))))

        # The recognized text is stored in variable text
        # Any string processing may be applied on text
        # Here, basic formatting has been done:
        # In many PDFs, at line ending, if a word can't
        # be written fully, a 'hyphen' is added.
        # The rest of the word is written in the next line
        # Eg: This is a sample text this word here GeeksF-
        # orGeeks is half on first line, remaining on next.
        # To remove this, we replace every '-\n' to ''.
        text = text.replace('-\n', '')  
        
        # Removing Image after reading
        os.remove(filename)

        # Finally, write the processed text to the file.
        f.write(text)

    # Close the file after writing all the text.
    f.close()    
    
    return ocr_file

In [5]:
def extract_headings(text):
    length = len(text)
    heading = ['title', 'abstract']
    sub_heading = heading.copy()
    number = 1
    for i in range(1, length):
        if(text[i] == 'References'):
            break
        if(text[i-1] != "" or text[i]==""):
            continue
        if(ord(text[i][0]) == number+48 or ord(text[i][0]) == number+47):
            tokens = text[i].split()
            if(len(tokens) >= 10):
                continue
            if(((len(tokens[0]) == 2 and tokens[0][1] == '.') or (len(tokens[0]) == 1))
            and tokens[0][0] == str(number)):
                head = " ".join(tokens[1:])
                heading.append(head)
                sub_heading.append(head)
                number += 1
            if((len(tokens[0]) == 4 and tokens[0][1] == '.' and tokens[0][3] == '.') or
               (len(tokens[0]) == 3 and tokens[0][1] == '.')):
                head = " ".join(tokens[1:])
                sub_heading.append(head)
    return heading, sub_heading

In [6]:
no_heading_list = []

In [8]:
path = "../NCG_Dataset/test-data/*/*"
directory = glob.glob(path)
#print(len(directory))
for dir_path in directory:
    start_time = time.time()
    file = "".join((dir_path,"/main_heading.txt"))
    #if(dir_path == "../NCG_Dataset/test-data/data-to-text_generation/5" or
    #   dir_path == "../NCG_Dataset/test-data/data-to-text_generation/6"):
    #    continue
    if(os.path.isfile(file)):
        continue
    else:
        print(dir_path)
        continue
    print(dir_path)
    file = "".join((dir_path,"/*.pdf"))
    pdf_file = glob.glob(file)[0]
    #print(pdf_file)
    pages = pdf_image(pdf_file)
    #print(pages)
    ocr_file = image_ocr(dir_path, pages)
    #print(ocr_file)
    #ocr_file = "../NCG_Dataset/training-data-master/natural_language_inference/1/ocr.txt"
    f = open(ocr_file, 'r')
    text = f.readlines()
    text = [line.strip() for line in text]
    #print(len(text))
    heading, sub_heading = extract_headings(text)
    file = "".join((dir_path,"/main_heading.txt"))
    f = open(file, 'w')
    for head in heading:
        f.write(head+"\n")
    f.close()
    file = "".join((dir_path,"/sub_heading.txt"))
    f = open(file, 'w')
    for head in sub_heading:
        f.write(head+"\n")
    f.close()
    end_time = time.time()
    if(len(heading) == 2):
        no_heading_list.append(dir_path+"\n")
f = open("trial_missing_heading.txt", 'w')
f.writelines(no_heading_list)
f.close()

../NCG_Dataset/test-data/data-to-text_generation/5
../NCG_Dataset/test-data/data-to-text_generation/6
